In [5]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from helper import *
from torchsummary import summary
import models

In [6]:
encoders = models.encoders.get_encoder_names()
print(encoders)

['resnet10', 'resnet14', 'resnet18', 'resnet20', 'resnet26', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnext101_32x16d', 'resnext101_32x32d', 'resnext101_32x48d', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'inceptionresnetv2', 'inceptionv4', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6', 'efficientnet-b7', 'mobilenet_v2', 'xception']


In [10]:
unet = models.unet.Unet('resnet10', classes = 11, encoder_weights = None).cuda()
# print(unet)
print(summary(unet, input_size = (3, 480, 640)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 320]           9,408
       BatchNorm2d-2         [-1, 64, 240, 320]             128
              ReLU-3         [-1, 64, 240, 320]               0
         MaxPool2d-4         [-1, 64, 120, 160]               0
            Conv2d-5         [-1, 64, 120, 160]          36,864
       BatchNorm2d-6         [-1, 64, 120, 160]             128
              ReLU-7         [-1, 64, 120, 160]               0
            Conv2d-8         [-1, 64, 120, 160]          36,864
       BatchNorm2d-9         [-1, 64, 120, 160]             128
             ReLU-10         [-1, 64, 120, 160]               0
       BasicBlock-11         [-1, 64, 120, 160]               0
           Conv2d-12          [-1, 128, 60, 80]          73,728
      BatchNorm2d-13          [-1, 128, 60, 80]             256
             ReLU-14          [-1, 128,

In [11]:
class SaveFeatures :
    def __init__(self, m) : 
        self.handle = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, m, inp, outp) : 
        self.features = outp
    def remove(self) :
        self.handle.remove()

In [12]:
sf = [SaveFeatures(m) for m in [unet.encoder.relu, 
                                unet.encoder.layer1, 
                                unet.encoder.layer2, 
                                unet.encoder.layer3, 
                                unet.encoder.layer4, 
                                unet.decoder.blocks[0], 
                                unet.decoder.blocks[1], 
                                unet.decoder.blocks[2], 
                                unet.decoder.blocks[3], 
                                unet.decoder.blocks[4] 
                               ]]

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.35675976, 0.37380189, 0.3764753], std = [0.32064945, 0.32098866, 0.32325324])
])
trainset = KITTI(split = 'train', transform = transform)
testset = KITTI(split = 'test', transform = transform)

trainloader = DataLoader(trainset, batch_size = 2, shuffle = True)
testloader = DataLoader(testset, batch_size = 1, shuffle = True)

In [16]:
# x, _ = next(iter(trainloader))
x = torch.randn(2, 3, 256, 1024)
x = x.cuda()
out = unet(x)
for i in range(10) :
    print(sf[i].features.shape)

torch.Size([2, 64, 128, 512])
torch.Size([2, 64, 64, 256])
torch.Size([2, 128, 32, 128])
torch.Size([2, 256, 16, 64])
torch.Size([2, 512, 8, 32])
torch.Size([2, 256, 16, 64])
torch.Size([2, 128, 32, 128])
torch.Size([2, 64, 64, 256])
torch.Size([2, 32, 128, 512])
torch.Size([2, 16, 256, 1024])


In [55]:
unet = models.unet.Unet('resnet18', classes = 19, activation = 'softmax', encoder_weights = None).cuda()
# print(unet)
print(summary(unet, input_size = (3, 512, 512)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           9,408
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1

In [56]:
# x, _ = next(iter(trainloader))
x = torch.randn(2, 3, 256, 1024)
x = x.cuda()
out = unet(x)
for i in range(10) :
    print(sf[i].features.shape)

torch.Size([2, 64, 128, 512])
torch.Size([2, 64, 64, 256])
torch.Size([2, 128, 32, 128])
torch.Size([2, 256, 16, 64])
torch.Size([2, 512, 8, 32])
torch.Size([2, 256, 16, 64])
torch.Size([2, 128, 32, 128])
torch.Size([2, 64, 64, 256])
torch.Size([2, 32, 128, 512])
torch.Size([2, 16, 256, 1024])


In [17]:
unet = models.unet.Unet('resnet34', classes = 19, activation = 'softmax').cuda()
unet2 = models.unet.Unet('resnet18', classes = 19, activation = 'softmax').cuda()

sf = [SaveFeatures(m) for m in [unet.encoder.relu, 
                                unet.encoder.layer1, 
                                unet.encoder.layer2, 
                                unet.encoder.layer3, 
                                unet.encoder.layer4, 
                                unet.decoder.blocks[0], 
                                unet.decoder.blocks[1], 
                                unet.decoder.blocks[2], 
                                unet.decoder.blocks[3], 
                                unet.decoder.blocks[4] 
                               ]]

sf2 = [SaveFeatures(m) for m in [unet2.encoder.relu, 
                                unet2.encoder.layer1, 
                                unet2.encoder.layer2, 
                                unet2.encoder.layer3, 
                                unet2.encoder.layer4, 
                                unet2.decoder.blocks[0], 
                                unet2.decoder.blocks[1], 
                                unet2.decoder.blocks[2], 
                                unet2.decoder.blocks[3], 
                                unet2.decoder.blocks[4] 
                               ]]

In [18]:
x = torch.randn(2, 3, 256, 1024).cuda()
_ = unet(x)
_ = unet2(x)
for i in range(10) :
    print(sf[i].features.shape)
    assert(sf[i].features.shape == sf2[i].features.shape)

torch.Size([2, 64, 128, 512])
torch.Size([2, 64, 64, 256])
torch.Size([2, 128, 32, 128])
torch.Size([2, 256, 16, 64])
torch.Size([2, 512, 8, 32])
torch.Size([2, 256, 16, 64])
torch.Size([2, 128, 32, 128])
torch.Size([2, 64, 64, 256])
torch.Size([2, 32, 128, 512])
torch.Size([2, 16, 256, 1024])
